In [2]:
import pandas as pd
import numpy as np
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth
from annoy import AnnoyIndex

In [3]:
def recurse_playlist(id):
    r = sp.playlist(id)
    playlists.append(r)
    if 'tracks' in r:
        if 'next' in r['tracks']:
            if r['tracks']['next'] is not None:
                recurse_playlist(str.replace(r['tracks']['next'], 'https://api.spotify.com/v1/playlists/', ''))
    elif 'next' in r:
        if r['next'] is not None:
            recurse_playlist(str.replace(r['next'], 'https://api.spotify.com/v1/playlists/', ''))
    else:
        pass

In [4]:
def get_metadata(ids):
    metadata = list()
    # 50 limit on api call    
    for i in range(0,len(ids),50):
        meta = sp.tracks(ids[i:i+50])
        for data in meta['tracks']:
            artists = list()
            if data is not None:
                for artist in data['artists']:
                    artists.append(artist['id'])
                metadata.append({"song_id":data['id'],  
                                 "artists":artists, 
                                 "song_name":data['name'], 
                                 "album_id":data['album']['id'],
                                 "album_name":data['album']['name'],
                                 "popularity":data['popularity']})
    return pd.DataFrame(metadata)

In [5]:
def get_features(ids):
    features = []
    # 50 limit on api call
    for i in range(0,len(ids),50):
        audio_features = sp.audio_features(ids[i:i+50])
        for track in audio_features:
            if track is not None:
                features.append(track)
    
    return pd.DataFrame(features)

In [6]:
def recurse_saved_songs(offset=0):
    r = sp.current_user_saved_tracks(20, offset)
    for i in r['items']:
        songs.append({"id":i['track']['id'], "added_at":i['added_at']})
    if r['next']:
        offset+=20
        recurse_saved_songs(offset)

In [7]:
cid ="0deb154cdea34cfa9c50fc76938403b9" 
secret = "6aa3c3ee390d4421bdc6a860cf33c686"
username = "Ben"

In [9]:
credentials = SpotifyClientCredentials(
        client_id=cid,
        client_secret=secret)

In [10]:
sp = spotipy.Spotify(auth_manager=credentials)

In [48]:
user_ids = [
    'kirstendodo',
    '1260745480',
    '1255526461',
    'athenajiang',
    '1239664093',
    '1268639225',
    '22eja5lr4xyqrmdgb4jox74pi',
    'russellkim98',
    '1213055384',
    '12563360',
    '1218288200',
    '22vxpm5y7bws42rgecwwdtqzq',
    '2lb6yucjb2mmlbpqvv7f6tpa',
    'hahowie',
    '1233302457',
    'clairehuangg',
    'gmfrlife',
    'karawho',
    '1210134132',
    'gracehuang22',
    'sangeethaisrofl',
    '124028238',
    '1228739853',
    '1249053478',
    '1249402231',
    '1261634901',
    'jasonliao',
    '12101413836',
    '227lrr7ufyoef3thjhnzjcbhi',
    '1263186670',
    '126549512',
    '22tbkcbk3npj7dmozb5xcvegq',
    '1258256267',
    'zenbhang',
    '1249768840',
    '1251291991',
    '1293249673',
    '12184871363',
    '1225107231',
    '21jm5sg2qelg57la434krzpjy',
    '22n5d3jkyzx6ucmdh6stknedi'
]

In [49]:
playlist_ids = list()
for ui in user_ids:
    try:
        r = sp.user_playlists(ui, limit=50)
        for i in r['items']:
            playlist_ids.append(i['id'])
    except:
        print(ui, 'error')

HTTP Error for GET to https://api.spotify.com/v1/users/2lb6yucjb2mmlbpqvv7f6tpa/playlists with Params: {'limit': 50, 'offset': 0} returned 404 due to Not found.


2lb6yucjb2mmlbpqvv7f6tpa error


In [13]:
playlists = list()
for i in playlist_ids:
    recurse_playlist(str.replace(i, 'https://open.spotify.com/playlist/', ''))

In [14]:
ids = []
for p in playlists:  
    if 'tracks' in p:
        for item in p['tracks']['items']:
            try:
                if item['track']['id'] is not None:
                    ids.append(item['track']['id'])
            except:
                pass
    elif 'items' in p:
        for item in p['items']:
            try:
                if item['track']['id'] is not None:
                    ids.append(item['track']['id'])
            except:
                pass

In [15]:
df_metadata = get_metadata(ids)

In [17]:
df_features = get_features(ids)
# We probably just want to build a db with all of these song + features

# User Features

In [18]:
scope = 'user-library-read'

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id = "0deb154cdea34cfa9c50fc76938403b9",
                                               client_secret = secret,
                                               redirect_uri= 'http://127.0.0.1:5000/spotify/callback',
                                               scope=scope,
                                              open_browser=True))

In [ ]:
# There's this 
# GET https://api.spotify.com/v1/me/top/{type}


In [ ]:
# if you authorize this app, we should take your profile URL and 

In [19]:
songs = list()
recurse_saved_songs()

In [20]:
df_your_songs = pd.DataFrame(songs)

In [21]:
df_your_songs.head()

,id,added_at
0,5fWG4JDot8nHCte7gUdMUS,2021-10-09T22:39:45Z
1,4piGD6lbuj57rxausi0brN,2021-10-06T23:12:48Z
2,5RQmbhnkrt6llPrxqRsym4,2021-10-03T12:18:34Z
3,75sMhi81W5Tw2AGlOGATuy,2021-09-28T03:56:05Z
4,0OOT6JmtkcYidsuQ9Fh54i,2021-09-21T16:30:28Z


In [59]:
len(df_features)

57500

In [25]:
# index can just be the integer : song id mapping LMFAO
df = df_features.reset_index()

In [30]:
test = np.array(df_features[['danceability','energy','key','loudness','mode','speechiness','acousticness','instrumentalness','liveness','valence','tempo']])

In [ ]:
def build_annoy_index(trees=10, array):
    # vector size is feature length, idk wtf angular does
    t = AnnoyIndex(11, 'angular')
    for i in range(len(array)):
        t.add_item(i, test[i])
    # once this is built, you can't add items to it. Should we build it somewhere else?
    t.build(trees)
    
    return t

In [ ]:
# select song ids that are not already in 
# worry about this later
# right now all we need to do is get the feature vectors for the songs you've loaded and check for nearest

In [1]:

# annoy is basically
# [1: [feature1, feature2, feature3]]
# it only supporst integers as identifiers for items, so will have to map
# t = AnnoyIndex(f, 'angular')  # Length of item vector that will be indexed
#     t.add_item(i, v)
# t.build(10) # 10 trees
# print(t.get_nns_by_item(0, 1000)) # will find the 1000 nearest neighbors

In [ ]:
def get_songs_for_nn():
# Apply some sampling criteria to get a bunch of songs based on the user's last X days of activity (sampling? most recent)
    
    return pass

In [60]:
def get_nn(t, songs):
    nn = list()
    song_features = np.array(get_features(songs)[['danceability','energy','key','loudness','mode','speechiness','acousticness','instrumentalness','liveness','valence','tempo']])
    for s in song_features:
        nn.append(t.get_nns_by_vector(v, 5, search_k=-1, include_distances=False))
    # for each song in songs, we get a few close songs. Return all of them?
    return nn

In [ ]:
def apply_filtering_criteria(songs_df):
    # stick a bunch of rules on there to get us to 30 songs

In [ ]:
def create_playlist(songs):
    # create the playlist for the user